In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'september'
new_month = 'october'

In [3]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')

In [4]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [5]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.index = map(lambda x: x.lower(), df.index)

In [6]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [7]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    if d_messari['data']['supply']['y_2050'] == None:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['circulating'])
    else:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Supply % Issued'] = df_all['Y2050 Supply'] / ((df_all['Y2050 Supply']))
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 180)
    except:
        pass
    return df_all

In [8]:
url_messari = "https://data.messari.io/api/v1/assets/elf/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)

In [9]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
    except:
        pass

In [10]:
metrics_log['elf']

Y2050 Supply         5.344801e+08
Supply % Issued      1.000000e+00
Price                7.680949e-02
Marketcap            4.105314e+07
Current Marketcap    4.105314e+07
Est Volume           2.912921e+08
dtype: float64

In [11]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [12]:
# t = 'ELF'
# metrics = get_messari_data(t)
# t = 'ELF_TEST'
# #print (metrics)
# metrics_log[t] = metrics
# metrics['Y2050 Supply']
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#    pass
# try:
#    df.loc[t,('Category2')] = metrics['Category2']
# except:
#    pass
# try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [13]:
# t = 'VET'
# metrics = get_messari_data(t)
# metrics_log[t] = metrics
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# #df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#     df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#     pass
# try:
#     df.loc[t,('Category2')] = metrics['Category2']
# except:
#     pass
# try:
#     df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [14]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [15]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [16]:
#read in other data sources
onchainfx_db = df

In [17]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

145
145
145
145


In [18]:
#fix any ticker mapping issues - rank date use these
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
asset_db = asset_db.rename({'IOTA' : 'MIOTA'})

In [19]:
# #fix any ticker mapping issues - rebalance day use these
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
# asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [20]:
#onchainfx_db['Marketcap'] = np.where(onchainfx_db['Marketcap'].isnull() == True,onchainfx_db['Current Marketcap'],onchainfx_db['Marketcap'])

In [21]:
onchainfx_db.loc['GNO']

Coin                      Gnosis
Class                          3
Sector                       NaN
Quality Exchange?              1
ERC20                          1
ICO ok?                        1
Eligible                       1
Y2050 Supply         1.10459e+06
Available Supply     1.10459e+06
Supply % Issued                1
Price                    14.5218
Marketcap            1.60406e+07
Current Marketcap    1.60406e+07
Est Volume           5.69122e+06
Name: GNO, dtype: object

In [22]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [23]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [24]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [25]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

In [26]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BTCP,2.562703e+07,2.562703e+07,1.0,2.101452e+07,1.219492,2.101452e+07,4.632615e+06,1,0.180771,1,0.0,1.0,0.0,2.0,0
CTXC,1.956314e+07,1.956314e+07,1.0,2.539879e+08,0.077024,2.539879e+08,3.730080e+08,1,19.066881,1,0.0,2.0,0.0,2.0,0
DENT,2.679525e+07,2.679525e+07,1.0,1.000000e+11,0.000268,1.000000e+11,3.720434e+08,1,13.884678,1,0.0,3.0,0.0,2.0,0
DRGN,8.238944e+06,8.238944e+06,1.0,2.384219e+08,0.034556,2.384219e+08,2.147579e+06,1,0.260662,1,0.0,3.0,0.0,2.0,0
ETN,3.968552e+07,3.968552e+07,1.0,9.831274e+09,0.004037,9.831274e+09,3.563027e+07,1,0.897815,1,0.0,3.0,0.0,2.0,0
KCS,9.820154e+07,9.820154e+07,1.0,8.780105e+07,1.118455,8.780105e+07,9.176384e+08,1,9.344441,1,0.0,2.0,0.0,2.0,0
R,2.289760e+07,2.289760e+07,1.0,4.844500e+08,0.047265,4.844500e+08,4.871846e+06,1,0.212767,1,0.0,3.0,0.0,2.0,0
RHOC,2.281999e+07,2.281999e+07,1.0,3.737899e+08,0.061050,3.737899e+08,2.443917e+06,1,0.107095,1,0.0,3.0,0.0,2.0,0
SMART,3.635644e+06,3.635644e+06,1.0,1.413859e+09,0.002571,1.413859e+09,6.653906e+05,1,0.183019,1,0.0,3.0,0.0,2.0,0
SPANK,2.844556e+06,2.844556e+06,1.0,3.000434e+08,0.009480,3.000434e+08,8.283023e+06,1,2.911886,1,0.0,3.0,1.0,2.0,0


In [27]:
eligible_db = eligible_db.append(ineligible_db.loc['XTZ'])
eligible_db = eligible_db.drop('ATOM')

In [28]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

In [29]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOT' : 'MIOTA'})
universe = universe.rename({'VEN' : 'VET'})
universe.loc['XTZ']

Marketcap            1.749287e+09
Current Marketcap    1.749287e+09
Float Percent        1.000000e+00
Float                1.923801e+09
Price                9.092869e-01
Total Supply         1.923801e+09
Volume               1.347448e+08
Float Pass           1.000000e+00
Volume Ratio         7.702841e-02
Volume Pass          0.000000e+00
Other Pass           1.000000e+00
Class                2.000000e+00
ERC20                0.000000e+00
Score                2.000000e+00
Eligible             0.000000e+00
Rank                 1.000000e+01
Name: XTZ, dtype: float64

In [30]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained ADA
Retained BNB
10 full! Do not add  LINK
10 full! Do not add  XTZ


In [31]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained BAT
Retained LSK
New add ZRX
New add QTUM
20 full! Do not add  BTG
20 full! Do not add  DGB
20 full! Do not add  SC
20 full! Do not add  KMD


In [32]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained WAN
Retained ELF
Retained PIVX
Retained WTC
Retained NAS
New add FUN
New add AION
New add CVC
New add RCN
New add KNC
40 full! Do not add  STRAT
40 full! Do not add  LRC
40 full! Do not add  RDD
40 full! Do not add  MANA
40 full! Do not add  DGD
40 full! Do not add  MET
40 full! Do not add  POLY
40 full! Do not add  PAY
40 full! Do not add  ENG
40 full! Do not add  BNT


In [33]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [34]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [35]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

Total adds ['AION' 'CVC' 'FUN' 'KNC' 'RCN']
Total deletes ['GNO' 'MANA' 'POWR' 'QKC' 'WAX']
Total count members: 70

40 adds ['AION' 'CVC' 'FUN' 'KNC' 'MAID' 'RCN']
40 deletes ['GNO' 'POWR' 'QKC' 'QTUM' 'WAX' 'ZRX']
40 count members: 40

20 adds ['QTUM' 'ZRX']
20 deletes ['MAID' 'MANA']
20 count members: 20

Application adds ['CVC' 'FUN' 'KNC' 'RCN']
Application deletes ['GNO' 'MANA' 'WAX']
Application count members: 24

Currency adds []
Currency deletes []
Currency count members: 19

Platform adds ['AION']
Platform deletes ['POWR' 'QKC']
Platform count members: 27

ETH adds ['AION' 'CVC' 'FUN' 'KNC']
ETH deletes ['GNO' 'MANA' 'POWR' 'WAX']
ETH count members: 20

10 adds []
10 deletes []
10 count members: 10

Sector count members: 70


In [37]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    if 'MIOTA' in indexes_new_tickers[k].index:
        indexes_new_tickers[k] = indexes_new_tickers[k].rename({'MIOTA' : 'IOT'})
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])